In [ ]:
# Implementation of the traveling salesman algorithms, sending the algorithm as an argument - strategy pattern.

import matplotlib.pyplot as mplot
import numpy as np
import math
import itertools
import networkx as nx

def traveling_salesman_problem(algorithm, graph, start, locations_to_ignore=None, type_of_output=None):
    # Parameters:
    # algorithm: the algorithm to use
    # graph: the graph to use
    # locations: an optional parameter, representing a list of locations to visit (if not given, the algorithm will use all the nodes in the graph)
    # type_of_output: an optional parameter, for a path to be returned as a list of nodes, for a length to be returned the length of the path.

    # Checks if the graph is in the form of a matrix or a networkx graph
    if type(graph) == np.ndarray:
        graph = nx.from_numpy_matrix(graph)
    elif type(graph) == nx.classes.graph.Graph:
        pass
    else:
        raise TypeError(
            "The graph must be in the form of a matrix or a networkx graph")
    # If the locations_to_ignore are given, it checks if they are in the graph
    if locations_to_ignore is not None:
        if start in locations_to_ignore:
            raise ValueError(
                "The start location cannot be in the locations to ignore")
        for location in locations_to_ignore:
            if location in graph.nodes:
                graph.remove_edges_from(graph.edges(location))
                graph.remove_node(location)
    # If the locations are not given, it uses all the nodes in the graph
    else:
        locations = graph.nodes
    # Checks if the algorithm is in the form of a function
    if not callable(algorithm):
        raise TypeError("The algorithm must be in the form of a function")
    # Checks if the type of output exists
    if type_of_output is not None:
        if type_of_output not in ["path", "length"]:
            raise ValueError(
                "The type of output must be either 'path' or 'length'")
    # Checks if the start node is in the graph
    if start not in graph.nodes:
        raise ValueError("The start node must be in the graph")

    # Naive algorithm
    def naive_algorithm(graph, start):
        end = start
        # Generates all the permutations of the nodes in the graph
        permutations = list(itertools.permutations(graph.nodes))
        # Calculates the cost of every permutation
        costs = {}
        for permutation in permutations:
            cost = 0
            for i in range(len(permutation) - 1):
                cost += graph[permutation[i]][permutation[i + 1]]["weight"]
            costs[cost] = permutation
        # Returns the permutation with the minimum cost
        return costs[min(costs.keys())]

    # Held-Karp dynamic algorithm
    def dynamic_algorithm(graph, start):
        # Generates all the subsets of the nodes in the graph
        subsets = []
        for i in range(1, len(graph.nodes)):
            subsets += list(itertools.combinations(graph.nodes, i))
        # Calculates the cost of every subset
        costs = {}
        for subset in subsets:
            for node in graph.nodes:
                if node not in subset:
                    costs[(node, subset)] = math.inf
        for node in graph.nodes:
            if node != start:
                costs[(node, ())] = graph[start][node]["weight"]
        # Calculates the cost of every subset
        for i in range(1, len(graph.nodes)):
            for subset in subsets:
                if len(subset) == i:
                    for node in graph.nodes:
                        if node not in subset:
                            costs[(node, subset)] = min(
                                [costs[(node, subset)] for node in graph.nodes if node not in subset] + [costs[(node, subset[0:i - 1])] + graph[node][subset[i - 1]]["weight"] for node in subset])
        # Returns the subset with the minimum cost
        return min([costs[(node, subsets[i])] + graph[node][start]["weight"] for i, node in enumerate(subsets[len(subsets) - 1])])

    operations = {"naive algorithm": naive_algorithm,
                  "dynamic algorithm": dynamic_algorithm}
    if type_of_output == "path":
        return operations[algorithm](graph, start)
    else:
        # Calculates the cost of the path
        cost = 0
        path = operations[algorithm](graph, start)
        for i in range(len(path) - 1):
            cost += graph[path[i]][path[i + 1]]["weight"]
        return cost